# Tensor Operations with Tensorflow


In [1]:
import tensorflow as tf
import numpy as np

## Tensor Product & Square & Sum

In [2]:

a = tf.placeholder(tf.float32, shape=(1, 3))
b = tf.placeholder(tf.float32, shape=(1, 2))
c = tf.placeholder(tf.float32, shape=(1, 3))
# two vectors outer dot -> a and b. 
ab = tf.tensordot(a, b, axes=([0], [0]))
# reshape the ab matrix into a 3D tensor
ab_ = tf.reshape(ab, shape=(3 ,2, 1))
# 2D matrix outer dot with the third vector
abc = tf.tensordot(ab_, c, axes=([2], [0])) 

# test the square for each element of vector
a_2 = tf.square(a)
# test the square for each element of matrix
ab_2 = tf.square(ab)
# calculate the sum of squared matrix
ab_2sum = tf.reduce_sum(ab_2, reduction_indices=[0,1])
# product the matrix with a constant
lamda = tf.Variable(2.0, tf.float32)
ab_lamda = lamda*ab


with tf.Session() as session:
    # initialize the variables
    session.run(tf.global_variables_initializer())
    # print ab
    print ((ab.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]})))
    # print ab_
    print ((ab_.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]})))
    # print abc
    X = abc.eval(
        feed_dict={ a: [[1,2,3]], b: [[5,6]] , c:[[1,1,1]]}
    )
    # print ab_2
    print ab_2.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})
    # print ab_2sum
    print ab_2sum.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})
    # print ab_lamda
    print ab_lamda.eval(feed_dict={a:[[1,2,3]], b:[[5,6]]})



[[  5.   6.]
 [ 10.  12.]
 [ 15.  18.]]
[[[  5.]
  [  6.]]

 [[ 10.]
  [ 12.]]

 [[ 15.]
  [ 18.]]]
[[  25.   36.]
 [ 100.  144.]
 [ 225.  324.]]
854.0
[[ 10.  12.]
 [ 20.  24.]
 [ 30.  36.]]


## Filter the NAN Value

In [3]:
A = [1,1,np.nan]

In [4]:
x = tf.placeholder(tf.float64, shape=[None, None])
nanX = tf.is_nan(x)
y = tf.boolean_mask(x,tf.logical_not(nanX),name='boolean_mask')
with tf.Session() as session:
    print ((nanX.eval(
        feed_dict={ x: [[np.nan,6]]})))
    print (y.eval(feed_dict={x:[[np.nan, 6],[4, np.nan]]}))

[[ True False]]
[ 6.  4.]


## 2D matrix Dot Product with Tensor in k-mode

In [5]:
matrix2D = tf.placeholder(tf.float32, shape=(1, 3))
tensor = tf.placeholder(tf.float32, shape = (1,2,3))

In [6]:
dotProduct = tf.tensordot(tensor, matrix2D, axes=([2], [1])) 

In [13]:
mat = np.array([[1,1,2]])

In [14]:
ts = np.array([[[1,1,1],[1,1,1]]])

In [15]:
ts.shape

(1, 2, 3)

In [18]:
with tf.Session() as session:
    print np.shape(dotProduct.eval(feed_dict={matrix2D:mat, tensor:ts}))

(1, 2, 1)



## SVD in tensorflow

In [86]:
import tensorflow as tf
import numpy as np
a = np.array([[1,2,3],[3,4,5]])
a = a.astype(np.float64)
s, u, v = tf.svd(a)
tf_a_approx = tf.matmul(u, tf.matmul(tf.diag(s), v, adjoint_b=True))
with tf.Session() as session:
    print tf_a_approx.eval()
    print s.eval()
    print u.eval()
    print v.eval()
u, s, v_adj = np.linalg.svd(a, full_matrices=False)
np_a_approx = np.dot(u, np.dot(np.diag(s), v_adj))

In [33]:
v_adj

array([[-0.39133557, -0.5605708 , -0.72980603],
       [ 0.8247362 ,  0.13817999, -0.54837623]])

In [34]:
np_a_approx

array([[ 1.,  2.,  3.],
       [ 3.,  4.,  5.]])

In [95]:
a = np.array([[[1,2,3,4],[3,4,5,6]],[[1,2,3,4],[3,4,5,6]]])
a = a.astype(np.float64)
s, u, v = tf.svd(a)
with tf.Session() as session:
    print u.eval().shape
print a.shape

(2, 2, 2)
(2, 2, 4)


## Flatten Tensor

In [48]:
a = np.random.rand(4,3,2)

In [54]:
for i in range(4):
    for j in range(3):
        for k in range(2):
            a[i,j,k] = (i+1)*100+(j+1)*10+(k+1)

In [55]:
a

array([[[ 111.,  112.],
        [ 121.,  122.],
        [ 131.,  132.]],

       [[ 211.,  212.],
        [ 221.,  222.],
        [ 231.,  232.]],

       [[ 311.,  312.],
        [ 321.,  322.],
        [ 331.,  332.]],

       [[ 411.,  412.],
        [ 421.,  422.],
        [ 431.,  432.]]])

In [60]:
shape = [4,3,2]
inputPlaceholder = tf.placeholder(tf.float64, shape=[4,3,2])

In [84]:
tf.unstack(inputPlaceholder, axis=2)

[<tf.Tensor 'unstack_15:0' shape=(4, 3) dtype=float64>,
 <tf.Tensor 'unstack_15:1' shape=(4, 3) dtype=float64>]

In [79]:
axes1Unroll = tf.concat(tf.unstack(inputPlaceholder, axis=0), axis=1)

In [81]:
with tf.Session() as session:
    print axes1Unroll.eval(feed_dict={inputPlaceholder:a})

[[ 111.  112.  211.  212.  311.  312.  411.  412.]
 [ 121.  122.  221.  222.  321.  322.  421.  422.]
 [ 131.  132.  231.  232.  331.  332.  431.  432.]]


In [82]:
axes0Unroll = tf.concat(tf.unstack(inputPlaceholder, axis=2), axis=1)

<tf.Tensor 'concat_6:0' shape=(4, 6) dtype=float64>